# Solving the PDE $\partial_t u - \Delta u = f$ with discontinuous Galerkin in time and continuous Galerkin in space in FEniCS by exploiting the kronecker product structure of the space-time matrix

In [1]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

Using discontinuous finite elements in time and continuous finite elements in space, the variational formulation for the PDE reads
$$
\sum_{m = 1}^M \int_{t_{m-1}}^{t_m} (\partial_t u, \phi) + (\nabla u, \nabla \phi)\ \mathrm{d}t + \sum_{m = 1}^{M-1} ([u]_m, \phi_m^+) + (u_0^+, \phi_0^+) = (u^0, \phi_0^+).
$$
Here the jump $[u]_m$ is defined as $[u]_m = u_m^+ - u_m^-$, where $u_m^+$ is the limit from above and $u_m^-$ is the limit from below at $t_m$.
The second term with the jump terms can be thought of as the integral/sum over the interior faces, since we have the temporal elements $(t_0, t_1), (t_1, t_2), ..., (t_{M-1}, t_M)$. Furthermore, we denote by the inner product $(f,g)$ the $L^2(\Omega)$ inner product, i.e.
$$
(f,g) := (f,g)_{L^2(\Omega)} := \int_\Omega f \cdot g\ \mathrm{d}x.
$$

In [2]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time

cpu_start_time = time.time()

set_log_active(False)
parameters["reorder_dofs_serial"] = False

We now define a helper function that takes in the temporal and spatial coordinates and returns its cartesian product. Let us explain how this function works with an example:
$$
\text{time_coords} = \begin{pmatrix} t_1\\ t_2\\ t_3 \end{pmatrix}, \quad
\text{space_coords} = \begin{pmatrix} X_1\\ X_2\\ X_3\\ X_4 \end{pmatrix} = \begin{pmatrix} x_1 & y_1\\ x_2& y_2\\ x_3& y_3\\ x_4& y_4 \end{pmatrix}
$$
Then the cartesian product of time_coords and space_coords is given by
$$
\begin{pmatrix} t_1 & X_1\\ t_1 & X_2\\ t_1 & X_3\\ t_1 & X_4 \\t_2 & X_1\\ t_2 & X_2\\ t_2 & X_3\\ t_2 & X_4 \\t_3 & X_1\\ t_3 & X_2\\ t_3 & X_3\\ t_3 & X_4 \\\end{pmatrix}
= \begin{pmatrix} t_1 & x_1 & y_1\\ t_1 & x_2 & y_2\\ t_1 & x_3 & y_3\\\ t_1 & x_4 & y_4 \\t_2 & x_1 & y_1\\ t_2 & x_2 & y_2\\ t_2 & x_3 & y_3\\ t_2 & x_4 & y_4 \\t_3 & x_1 & y_1\\ t_3 & x_2 & y_2\\ t_3 & x_3 & y_3\\ t_3 & x_4 & y_4 \\\end{pmatrix}
$$

In [3]:
def cartesian_product_coords(time_coords, space_coords):
    long_time_coords = np.kron(time_coords, np.ones((space_coords.shape[0],1)))
    long_space_coords = np.kron(np.ones((time_coords.shape[0],1)), space_coords)
    return np.hstack((long_time_coords, long_space_coords))

To make the space-time assembly more readable, we define the classes `TimeFE` and `SpaceFE` whose objects under the hood will take care of the temporal and spatial FEM matrix assemblies for us. The main observation for the variational formulation from above is for the assembly we can clearly separate spatial and temporal contributions. More concretely, each space-time test function $\phi(t,x)$ can be written as a product of a temporal basis function $\phi^k(t)$ and a spatial basis function $\phi^h(x)$. Therefore the finite element solution is given by 
$$
u(t,x) = \sum_{i_k = 1}^{\# DoF(\mathcal{T}_k)} \sum_{i_h = 1}^{\# DoF(\mathcal{T}_h)} U_{i_k, i_h}\phi^k_{i_k}(t)\phi^h_{i_h}(x).
$$
This separation of variables idea also can be applied to the space-time FEM matrices, e.g. we have
$$
\int_0^T \int_\Omega \partial_t \Big(\phi^k_{j_k}(t)\phi^h_{j_h}(x)\Big) \cdot (\phi^k_{i_k}(t)\phi^h_{i_h}(x))\ \mathrm{d}x \ \mathrm{d}t = \Big(\int_0^T  \partial_t \phi^k_{j_k}(t) \phi^k_{i_k}(t)\ \mathrm{d}t\Big) \cdot \Big(\int_\Omega \phi^h_{j_h}(x)\phi^h_{i_h}(x) \ \mathrm{d}x \Big)      
$$
and
$$
\int_0^T \int_\Omega \nabla_x \Big(\phi^k_{j_k}(t)\phi^h_{j_h}(x)\Big) \cdot \nabla_x\Big(\phi^k_{i_k}(t)\phi^h_{i_h}(x)\Big)\ \mathrm{d}x \ \mathrm{d}t = \Big(\int_0^T  \phi^k_{j_k}(t) \phi^k_{i_k}(t)\ \mathrm{d}t\Big) \cdot \Big(\int_\Omega \nabla_x\phi^h_{j_h}(x) \cdot \nabla_x\phi^h_{i_h}(x) \ \mathrm{d}x \Big).    
$$
Here, we notice that e.g. the matrix for the latter integral can be assmbled as $M_k \otimes K_h$, where $M_k$ is the temporal mass matrix, $K_h$ is the spatial stiffness/laplace matrix and $\otimes$ denotes the Kronecker product, i.e. for $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{p \times q}$ we have
$$
    A \otimes B :=  \begin{pmatrix}
        A_{11}B & \cdots & A_{1n}B \\
        \vdots & \ddots & \vdots \\
        A_{m1}B & \cdots & A_{mn}B
    \end{pmatrix} \in \mathbb{R}^{mp \times nq}.
$$

We start off by explaining the class `SpaceFE` which is very close to what we now to the FEM assembly for stationary problems. The constructor takes in a `mesh` and a function space `V`. With this we can assemble the spatial mass matrix ($M_h$) and stiffness/laplace matrix $K_h$, which we then convert to `scipy.sparse`-matrices.

In [4]:
class SpaceFE:
    form = {}
    matrix = {}

    def __init__(self, mesh, V):
        self.mesh = mesh
        self.V = V
        self.dofs = self.V.tabulate_dof_coordinates().reshape((-1, mesh.geometry().dim()))
        self.n_dofs = self.dofs.shape[0]
        # For debugging:
        # self.print_dofs()
        self.assemble_matrices()

    def print_dofs(self):
        print("\nSpace DoFs:")
        for dof, dof_x in zip(self.V.dofmap().dofs(), self.dofs):
          print(dof, ':', dof_x)

    def assemble_matrices(self):
        u = TrialFunction(self.V)
        phi = TestFunction(self.V)

        self.form["laplace"] = inner(grad(u), grad(phi))*dx
        self.form["mass"] = u*phi*dx

        for (name, _form) in self.form.items():
          self.matrix[name] = scipy.sparse.csr_matrix(
              as_backend_type(assemble(_form)).mat().getValuesCSR()[::-1],
              shape=(self.n_dofs, self.n_dofs)
          )

The class `TimeFE` works in the same way as its spatial counterpart, but we now need to deal with the discontinuity between temporal elements which leads to additional terms. We again assemble the temporal mass matrix ($M_k$) and a matrix which we denote by the temporal derivative matrix ($\tilde{C_k}$), because it includes all terms that are related to the temporal derivative in the variational formulation. This also includes the jump terms and the left hand side of the enforcement of the initial condition. With this the space-time system matrix is defined as $\tilde{C_k} \otimes M_h + M_k \otimes K_h$.

In [5]:
class TimeFE:
    form = {}
    matrix = {}

    def __init__(self, mesh, V):
        self.mesh = mesh
        self.V = V
        self.dofs = self.V.tabulate_dof_coordinates().reshape((-1, 1))
        self.n_dofs = self.dofs.shape[0]
        # For debugging:
        # self.print_dofs()
        self.assemble_matrices()

    def print_dofs(self):
        print("\nTime DoFs:")
        for dof, dof_t in zip(self.V.dofmap().dofs(), self.dofs):
            print(dof, ':', dof_t)

    def assemble_matrices(self):
        initial_time = CompiledSubDomain('near(x[0], t0)', t0=self.dofs[0,0])
        interior_facets = CompiledSubDomain("!on_boundary")
        boundary_marker = MeshFunction("size_t", self.mesh, 0)
        boundary_marker.set_all(0)
        initial_time.mark(boundary_marker, 1)
        interior_facets.mark(boundary_marker, 2)

        # Measure for the initial time
        d0 = Measure('ds', domain=self.mesh, subdomain_data=boundary_marker, subdomain_id=1)
        dS = Measure('dS', domain=self.mesh, subdomain_data=boundary_marker, subdomain_id=2)

        u = TrialFunction(self.V)
        phi = TestFunction(self.V)

        # NOTE: FEniCS has weird definitions for '+' and '-' (https://fenicsproject.discourse.group/t/integrating-over-an-interior-surface/247/3)
        self.form["derivative"] = grad(u)[0]*phi*dx + (u('-')-u('+'))*phi('-')*dS + u('+')*phi('+')*d0
        self.form["mass"] = u*phi*dx

        for (name, _form) in self.form.items():
            self.matrix[name] = scipy.sparse.csr_matrix(
              as_backend_type(assemble(_form)).mat().getValuesCSR()[::-1],
              shape=(self.n_dofs, self.n_dofs)
            )

We start the simulation by creating a spatial mesh and function space, as well as an object of type `SpaceFE`.

In [6]:
##############
# SIMULATION #
##############
L2_error = 0.
total_temporal_dofs = 0

###########################################
# 1. Create spatial finite element object #
###########################################
s = 1 # polynomial degree in space
DIM = 1
# DIM = 2 #-> Focussing on 1+1D for simplicity but DIM = 2 and DIM = 3 also possible with minor modifications
space_mesh = None
if DIM == 1:
    n_x = 1500 # number of spatial elements in x-direction
    space_mesh = IntervalMesh(n_x, 0., 1.) # \Omega = (0,1)
elif DIM == 2:
    n_x = 10 # number of spatial elements in x-direction
    n_y = 10 # number of spatial elements in y-direction
    space_mesh = UnitSquareMesh(n_x, n_y) # \Omega = (0,1) x (0,1)
else:
    raise NotImplementedError(f"dimension = {DIM} is not supported")
space_V = FunctionSpace(space_mesh, 'CG', s)

####################
# Spatial FE object:
Space = SpaceFE(space_mesh, space_V)

For the simulation, we now define the temporal element, the size of the time slabs. Then we create functions for the analytical solution and the right hand side (rhs) function $f$. Finally, we create an auxillary vector which is $0$ if the spatial DoF is inside the domain and $1$ if the spatial DoF is on the boundary.

In [7]:
#################################################
# 2. Start a time marching / time slabbing loop #
#################################################
start_time = 0.
end_time = 0.5
slab_size = 0.01 #0.05 # 0.25
slabs = [(start_time, start_time+slab_size)]
while slabs[-1][1] < end_time - 1e-8:
    slabs.append((slabs[-1][1], slabs[-1][1]+slab_size))
#print(f"\nSlabs = {slabs}")

# define analytical solution and rhs function
def analytical_solution_1D(X):
    _x = X[:, 1]
    _t = X[:, 0]
    return np.sin(np.pi*_x)*(1.+_t)*np.exp(-0.5*_t)

def rhs_function_1D(X):
    _x = X[:, 1]
    _t = X[:, 0]
    return np.sin(np.pi*_x)*np.exp(-0.5*_t)*(0.5+np.pi**2+(np.pi**2-0.5)*_t)

if DIM == 1:
    analytical_solution = analytical_solution_1D
    rhs_function = rhs_function_1D
else:
    raise NotImplementedError(f"Model problem for {DIM}+1D has not been implemented yet.")

# compute initial condition == analytical solution at t = start_time
u0 = analytical_solution(cartesian_product_coords(np.array([[start_time]]), Space.dofs))

# compute spatial boundary DoF indices
bc = DirichletBC(Space.V, Constant(1.), lambda _, on_boundary: on_boundary)
Space.boundary_dof_vector = np.zeros((Space.n_dofs,))
for i in bc.get_boundary_values().keys():
    Space.boundary_dof_vector[i] = 1.

Now, we are ready for the time marching loop, where we solve forward in time on each time slab.

In [8]:
#####################
# Time slabbing loop:
for i, slab in enumerate(slabs):
    print(f"Solving on slab_{i} = Ω x ({round(slab[0],5)}, {round(slab[1],5)}) ...")

    #############################################
    # 2a) Create temporal finite element object #
    #############################################
    r = 1 # polynomial degree in time
    n_time = 4 # number of temporal elements
    time_mesh = IntervalMesh(n_time, slab[0], slab[1]) # Start time: slab[0], End time: slab[1] = slab[0]+slab_size
    time_V = FunctionSpace(time_mesh, 'DG', r)

    #####################
    # Temporal FE object:
    Time = TimeFE(time_mesh, time_V)
    total_temporal_dofs += Time.n_dofs

    ####################################
    # 2b) Assemble space-time matrices #
    ####################################
    # NOTE: For linear PDEs this is just the kronecker product of temporal and spatial matrices
    system_matrix = scipy.sparse.kron(Time.matrix["derivative"], Space.matrix["mass"]) + scipy.sparse.kron(Time.matrix["mass"], Space.matrix["laplace"])
    mass_matrix = scipy.sparse.kron(Time.matrix["mass"], Space.matrix["mass"])

    #######################################
    # 2c) Assemble right hand side vector #
    #######################################
    # evaluate rhs function at the space-time Dofs and define the RHS as the space-time mass matrix times the projection of the RHS function
    space_time_coords = cartesian_product_coords(Time.dofs, Space.dofs)
    rhs = mass_matrix.dot(rhs_function(space_time_coords))
    # add weak imposition of the initial condition to the right hand side
    rhs[:Space.n_dofs] += Space.matrix["mass"].dot(u0)

    #################################
    # 2d) Apply boundary conditions #
    #################################
    # set the analytical solution as Dirichlet boundary conditions on the entire spatial boundary
    dofs_at_boundary =  np.kron(np.ones((Time.dofs.shape[0],1)), Space.boundary_dof_vector.reshape(-1,1)).flatten()
    slab_analytical_solution = analytical_solution(space_time_coords)

    # apply space-time BC to system matrix
    system_matrix = system_matrix.multiply((1.-dofs_at_boundary).reshape(-1,1)) + scipy.sparse.diags(dofs_at_boundary)

    # apply space-time BC to right hand side
    rhs = rhs * (1. - dofs_at_boundary) + slab_analytical_solution * dofs_at_boundary

    ###########################
    # 2e) Solve linear system #
    ###########################
    # solve the linear system with a sparse direct solver
    slab_solution = scipy.sparse.linalg.spsolve(system_matrix, rhs)

    ############################################
    # 2f) Compute error to analytical solution #
    ############################################
    # compute the error_vector between the projected analytical solution and the FEM solution
    slab_error = slab_analytical_solution - slab_solution
    # using the space-time mass matrix compute the L^2(I, L^2(Ω)) error on the slab
    L2_error += mass_matrix.dot(slab_error).dot(slab_error)

    # prepare next slab: get initial condition on next slab as final condition from this slab
    last_time_dof = Time.dofs.argmax()
    u0 = slab_solution[last_time_dof*Space.n_dofs:(last_time_dof+1)*Space.n_dofs]
    print("Done.\n")

Solving on slab_0 = Ω x (0.0, 0.01) ...
Done.

Solving on slab_1 = Ω x (0.01, 0.02) ...
Done.

Solving on slab_2 = Ω x (0.02, 0.03) ...
Done.

Solving on slab_3 = Ω x (0.03, 0.04) ...
Done.

Solving on slab_4 = Ω x (0.04, 0.05) ...
Done.

Solving on slab_5 = Ω x (0.05, 0.06) ...
Done.

Solving on slab_6 = Ω x (0.06, 0.07) ...
Done.

Solving on slab_7 = Ω x (0.07, 0.08) ...
Done.

Solving on slab_8 = Ω x (0.08, 0.09) ...
Done.

Solving on slab_9 = Ω x (0.09, 0.1) ...
Done.

Solving on slab_10 = Ω x (0.1, 0.11) ...
Done.

Solving on slab_11 = Ω x (0.11, 0.12) ...
Done.

Solving on slab_12 = Ω x (0.12, 0.13) ...
Done.

Solving on slab_13 = Ω x (0.13, 0.14) ...
Done.

Solving on slab_14 = Ω x (0.14, 0.15) ...
Done.

Solving on slab_15 = Ω x (0.15, 0.16) ...
Done.

Solving on slab_16 = Ω x (0.16, 0.17) ...
Done.

Solving on slab_17 = Ω x (0.17, 0.18) ...
Done.

Solving on slab_18 = Ω x (0.18, 0.19) ...
Done.

Solving on slab_19 = Ω x (0.19, 0.2) ...
Done.

Solving on slab_20 = Ω x (0.2, 0.2

Finally, we give a summary of the simulation and print the number od degrees of freedom, the error between the FEM and the analytical solution and the total CPU time.

In [9]:
print("------------")
print("| RESULTS: |")
print("------------")
print(f"#Temporal Dofs: {total_temporal_dofs}")
print(f"#Spatial Dofs: {Space.n_dofs}")
print(f"Space-time Dofs: {total_temporal_dofs * Space.n_dofs}")
print("L2 error:", L2_error)
print(f"CPU Time: {round(time.time() - cpu_start_time, 5)} s")

------------
| RESULTS: |
------------
#Temporal Dofs: 400
#Spatial Dofs: 1501
Space-time Dofs: 600400
L2 error: 3.282747233075526e-14
CPU Time: 3.45415 s


<FONT COLOR="#ff0000"> HOMEWORK: How can you speed up the computation from above (but still use the same number of spatial and temporal DoFs)?</FONT>